In [ ]:
from vosk import Model, KaldiRecognizer, SpkModel
import matplotlib.pyplot as plt
%matplotlib inline
import wave
import csv
import os
from pydub import AudioSegment
from NGrams_TFIDF_SimCoseno import *

from wordcloud import WordCloud
import nltk
stopwords = nltk.corpus.stopwords.words('spanish')

import numpy as np
def cosine_dist(x, y):
    nx = np.array(x)
    ny = np.array(y)
    return 1 - np.dot(nx, ny) / (np.linalg.norm(nx) * np.linalg.norm(ny))

def totime(a):
    h = int(a/3600)
    a -= h*3600
    m = int(a/60)
    a -= m*60
    s = int(a)
    return '{:02.0f}:{:02.0f}:{:02.0f}'.format(h,m,s)

sw = ['pues']
for w in sw:
    stopwords.append(w)

# Grabar un audio

In [ ]:
import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 6
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()




In [ ]:
#convierte a mono las pistas en estereo
file='LongWelcome.wav'
wf = wave.open(file, 'rb')
if wf.getnchannels() != 1:
    sound = AudioSegment.from_wav(file)
    sound = sound.set_channels(1)
    sound.export(file[:-4]+'_mono.wav', format='wav')

# Transcribe una grabación

In [ ]:
spk_model_path = 'vosk-model-spk-0.4'
model_path     = 'vosk-model-small-es-0.3'
#model_path     = 'vosk-model-en-us-aspire-0.2'


path = '.\\waves\\'
file = 'Gema_Molina'
file = 'guitarra'

wf = wave.open(path+file+'.wav', 'rb')
rate     = wf.getframerate()
if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
    print ("Audio file must be WAV format mono PCM.")
    exit (1)

model = Model(model_path)
spk_model = SpkModel(spk_model_path)
rec = KaldiRecognizer(model, spk_model, wf.getframerate())

tim = '00:00:00'
transcript = []
with open(path+file+'.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = rec.FinalResult()
            text = eval(res)['text']
            print (tim, text)

            writer.writerow([tim, text+' '])

            transcript.append([tim, text])
            tim= totime(wf.tell()/rate)


# Analiza transcripción y busca coincidencias

In [ ]:
expresion = 'guitarra'
file      = 'guitarra'

transcript=[]
with open(path+file+'.csv', mode='r', newline='') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        tim, text = row[0], row[1]
        transcript.append([tim, text])

matches    = []
fulltext   = ''
print ('Buscando   "', expresion, '"\n')
for  tim, text in transcript:
    t, dist = Rebeca(expresion, text)
    if dist>65:
        matches.append([tim, text])
        print(tim, text, '\n')
    fulltext += text.capitalize() 

if len(matches)==0:
    print ('...No encontrado')

# Word Cloud

In [ ]:
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, 
                      background_color='lavender', colormap='Set2', 
                      collocations=True, stopwords = stopwords).generate(fulltext)

plt.figure(figsize=(30, 20))
plt.imshow(wordcloud) 
plt.axis("off")